In [1]:
!pip install /kaggle/input/dicomsdl--0-109-2/dicomsdl-0.109.2-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl

import sys
!cp -r '/kaggle/input/contrails-libraries/pretrainedmodels-0.7.4/' './'
#!pip install './pretrainedmodels-0.7.4/pretrainedmodels-0.7.4/'
sys.path.append('./pretrainedmodels-0.7.4/pretrainedmodels-0.7.4/')

!cp -r '/kaggle/input/contrails-libraries/efficientnet_pytorch-0.7.1/' './'
#!pip install './efficientnet_pytorch-0.7.1/efficientnet_pytorch-0.7.1/'
sys.path.append('./efficientnet_pytorch-0.7.1/efficientnet_pytorch-0.7.1/')

!pip install '/kaggle/input/contrails-libraries/segmentation_models_pytorch-0.3.3-py3-none-any.whl' --no-deps

!pip install /kaggle/input/contrails-model-def1/einops-0.6.1-py3-none-any.whl

Processing /kaggle/input/dicomsdl--0-109-2/dicomsdl-0.109.2-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
Processing /kaggle/input/contrails-libraries/segmentation_models_pytorch-0.3.3-py3-none-any.whl
Processing /kaggle/input/contrails-model-def1/einops-0.6.1-py3-none-any.whl


In [2]:
%%writefile ddp.py

import sys
#!pip install './pretrainedmodels-0.7.4/pretrainedmodels-0.7.4/'
sys.path.append('./pretrainedmodels-0.7.4/pretrainedmodels-0.7.4/')

#!pip install './efficientnet_pytorch-0.7.1/efficientnet_pytorch-0.7.1/'
sys.path.append('./efficientnet_pytorch-0.7.1/efficientnet_pytorch-0.7.1/')

import dicomsdl
def __dataset__to_numpy_image(self, index=0):
    info = self.getPixelDataInfo()
    dtype = info['dtype']
    if info['SamplesPerPixel'] != 1:
        raise RuntimeError('SamplesPerPixel != 1')
    else:
        shape = [info['Rows'], info['Cols']]
    outarr = np.empty(shape, dtype=dtype)
    self.copyFrameData(index, outarr)
    return outarr
dicomsdl._dicomsdl.DataSet.to_numpy_image = __dataset__to_numpy_image   

import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
import os, copy, time

import cv2
from PIL import Image
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2

import pydicom

import torch
from torch import nn, optim
import timm
import segmentation_models_pytorch as smp

sys.path.append("/kaggle/input/rsna-abd-models-classes/")
from models import *


local_rank = int(os.environ['LOCAL_RANK'])
torch.cuda.set_device(local_rank)


def glob_sorted(path):
    return sorted(glob(path), key=lambda x: int(x.split('/')[-1].split('.')[0]))

def get_rescaled_image(dcm, img):
    resI, resS = dcm.RescaleIntercept, dcm.RescaleSlope
    
    img = resS * img + resI
    
    return img

def get_windowed_image(img, WL=50, WW=400):
    upper, lower = WL+WW//2, WL-WW//2
    X = np.clip(img.copy(), lower, upper)
    X = X - np.min(X)
    X = X / np.max(X)
    X = (X*255.0).astype('uint8')
    
    return X

def standardize_pixel_array(dcm, pixel_array):
    """
    Source : https://www.kaggle.com/competitions/rsna-2023-abdominal-trauma-detection/discussion/427217
    """
    # Correct DICOM pixel_array if PixelRepresentation == 1.
    #pixel_array = dcm.pixel_array
    
    if dcm.PixelRepresentation == 1:
        bit_shift = dcm.BitsAllocated - dcm.BitsStored
        dtype = pixel_array.dtype 
        pixel_array = (pixel_array << bit_shift).astype(dtype) >>  bit_shift
#         pixel_array = pydicom.pixel_data_handlers.util.apply_modality_lut(new_array, dcm)

    intercept = float(dcm.RescaleIntercept)
    slope = float(dcm.RescaleSlope)
    center = int(dcm.WindowCenter)
    width = int(dcm.WindowWidth)
    low = center - width / 2
    high = center + width / 2    
    
    pixel_array = (pixel_array * slope) + intercept
    pixel_array = np.clip(pixel_array, low, high)

    return pixel_array

def load_volume(dcms):
    volume = []
    file_to_volume_theo = {}
    pos_zs = []
    
    for dcm_path in dcms:
        pydcm = pydicom.dcmread(dcm_path)
        #image = dcm.pixel_array
        
        pos_z = pydcm[(0x20, 0x32)].value[-1]
        pos_zs.append(pos_z)
        
        dcm = dicomsdl.open(dcm_path)
        
        orig_image = dcm.to_numpy_image()
        
        image = get_rescaled_image(dcm, orig_image)
        image = get_windowed_image(image)
        
        image2 = standardize_pixel_array(dcm, orig_image)
        image2 = (image2 - image2.min()) / (image2.max() - image2.min() + 1e-6)
        
        if pydcm.PhotometricInterpretation == "MONOCHROME1":
            image2 = 1 - image2
        
        image2 = (image2 * 255).astype(np.uint8)
        
        file_to_volume_theo[dcm_path] = image2
        
        #image = get_windowed_image(dcm)
        
        if np.min(image)<0:
            image = image + np.abs(np.min(image))
        
        image = image / image.max()
        
        
        image = (image * 255).astype(np.uint8)
        
        
        volume.append(image)
    
    idxs = np.argsort(pos_zs)
    files_theo = np.array(dcms)[idxs]
    
    return np.stack(volume), file_to_volume_theo, files_theo


def load_volume_theo(dcms):
    volume = []
    imgs = {}
    for dcm_path in dcms:
        dicom = pydicom.dcmread(dcm_path)

        pos_z = dicom[(0x20, 0x32)].value[-1]
        
        dcm = dicomsdl.open(dcm_path)
        img = standardize_pixel_array(dcm)
        img = (img - img.min()) / (img.max() - img.min() + 1e-6)
        
        if dicom.PhotometricInterpretation == "MONOCHROME1":
            img = 1 - img
        
        img = (img * 255).astype(np.uint8)
        
        imgs[pos_z] = img
    
    for i, k in enumerate(sorted(imgs.keys())):
        img = imgs[k]
        
        volume.append(img)
    
    return np.stack(volume), np.array(dcms)[np.argsort(list(imgs.keys()))]

def process_volume(volume):
    volume = np.stack([cv2.resize(x, (128, 128)) for x in volume])
    
    volumes = []
    cuts = [(x, x+32) for x in np.arange(0, volume.shape[0], 32)[:-1]]
    
    if cuts:
        for cut in cuts:
            volumes.append(volume[cut[0]:cut[1]])
            
        volumes = np.stack(volumes)
    else:
        volumes = np.zeros((1, 32, 128, 128), dtype=np.uint8)
        volumes[0, :len(volume)] = volume
    
    if cuts:
        last_volume = np.zeros((1, 32, 128, 128), dtype=np.uint8)
        last_volume[0, :volume[cuts[-1][1]:].shape[0]] =  volume[cuts[-1][1]:]
    
        volumes = np.concatenate([volumes, last_volume])
    
    volumes = torch.as_tensor(volumes).float()
    
    return volumes

def get_volume_data(grd, step=96, stride=1, stride_cutoff=200):
    volumes = []
    
    if len(grd)>stride_cutoff:
        grd = grd[::stride]

    take_last = False
    if not str(len(grd)/step).endswith('.0'):
        take_last = True

    started = False
    for i in range(len(grd)//step):
        rows = grd[i*step:(i+1)*step]

        if len(rows)!=step:
            rows = pd.DataFrame([rows.iloc[int(x*len(rows))] for x in np.arange(0, 1, 1/step)])

        volumes.append(rows)

        started = True

    if not started:
        rows = grd
        rows = pd.DataFrame([rows.iloc[int(x*len(rows))] for x in np.arange(0, 1, 1/step)])
        volumes.append(rows)

    if take_last:
        rows = grd[-step:]
        if len(rows)==step:
            volumes.append(rows)

    return volumes



IMAGE_FOLDER = '/kaggle/input/rsna-2023-abdominal-trauma-detection/test_images/'

test_patients = np.array(os.listdir(IMAGE_FOLDER))

DEBUG = False
if len(test_patients)<10:
    DEBUG = True
    IMAGE_FOLDER = '/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/'

test_patients = np.array(os.listdir(IMAGE_FOLDER))

test_augs = A.Compose([
    A.Resize(384, 384),
    ToTensorV2()
])

test_augs2 = A.Compose([
    A.Resize(384, 384),
    ToTensorV2()
])

if DEBUG:
    submission = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/sample_submission.csv')
    
test_patients, test_patients.shape


        
        
        
        
segmentation_models = []
for F in [0]:
    model = convert_3d(SegmentationModel())
    state_dict = torch.load(f'/kaggle/input/rsna-abd-models/try3_seg_resnet18d_v3/zip/{F}.pth')
    model.load_state_dict(state_dict)
    model.eval()
    model.cuda()
    import copy
    segmentation_models.append(copy.deepcopy(model))
    
with torch.no_grad():
    outs = model(torch.zeros((2, 32, 128, 128)).cuda())
_ = [print(o.shape) for o in outs]

classification_models = []

'''
for F in [0, 1, 2]:
    path = f"/kaggle/input/coatlitemedium-384-exp1/{F}.pth"
    st = torch.load(path, map_location='cpu')
    
    model = Model4(num_classes=10, arch='medium', mask_head=False)
    model.load_state_dict(st)
    model.cuda()
    model.eval()
    
    classification_models.append(copy.deepcopy(model))
#'''

'''
for F in [1, 3]:
    path = f"/kaggle/input/coatmed-newseg-ourdata-4f/{F}.pth"
    st = torch.load(path, map_location='cpu')
    
    model = Model5(num_classes=10, arch='medium', mask_head=False)
    model.load_state_dict(st)
    model.cuda()
    model.eval()
    
    classification_models.append(copy.deepcopy(model))
#'''

'''
for F in range(1, 4):
    path = f"/kaggle/input/rsna-abd-models/try5_cls_tf_efficientnetv2_s_in21ft1k_v10/{F}.pth"
    st = torch.load(path, map_location='cpu')
   
    model = Model3(n_classes=10)
    model.load_state_dict(st)
    model.cuda()
    model.eval()

    classification_models.append(copy.deepcopy(model))
#'''

'''
for F in [3407, 69, 42, 17716124]:
    path = f"/kaggle/input/rsna-abd-models-2/try11_cls_tf_efficientnetv2_s_in21ft1k_v1_fulldata/{F}.pth"
    st = torch.load(path, map_location='cpu')
   
    model = Model3(n_classes=10)
    model.load_state_dict(st)
    model.cuda()
    model.eval()

    classification_models.append(copy.deepcopy(model))
#'''

#'''


#------------------------------------------------------------------------
#------------------------------------------------------------------------
#------------------------------------------------------------------------

for F in range(1):
    path = f"/kaggle/input/coatmed384ourdataseed100/3.pth"
    st = torch.load(path, map_location='cpu')

    model = Model4(num_classes=10, seg_classes=4, arch='medium', mask_head=False)
    model.load_state_dict(st)
    model.cuda()
    model.eval()

    classification_models.append(copy.deepcopy(model))

for F in range(1):
    path = f"/kaggle/input/coatmed384ourdataseed6969/3.pth"
    st = torch.load(path, map_location='cpu')

    model = Model4(num_classes=10, seg_classes=4, arch='medium', mask_head=False)
    model.load_state_dict(st)
    model.cuda()
    model.eval()

    classification_models.append(copy.deepcopy(model))
    
#for F in range(1):
#    path = f"/kaggle/input/coatmed384fullseed696969/3.pth"
#    st = torch.load(path, map_location='cpu')

#    model = Model4(num_classes=10, seg_classes=4, arch='medium', mask_head=False)
#    model.load_state_dict(st)
#    model.cuda()
#    model.eval()

#    classification_models.append(copy.deepcopy(model))
    
for F in range(1):
    path = f"/kaggle/input/coatmed384fullseed969696/3.pth"
    st = torch.load(path, map_location='cpu')

    model = Model4(num_classes=10, seg_classes=4, arch='medium', mask_head=False)
    model.load_state_dict(st)
    model.cuda()
    model.eval()

    classification_models.append(copy.deepcopy(model))
    
    
#------------------------------------------------------------------------

for F in [1]:
    path = f"/kaggle/input/coatmed-newseg-ourdata-4f/{F}.pth"
    st = torch.load(path, map_location='cpu')
    
    model = Model5(num_classes=10, arch='medium', mask_head=False)
    model.load_state_dict(st)
    model.cuda()
    model.eval()
    
    classification_models.append(copy.deepcopy(model))

    
for F in [2]:
    path = f"/kaggle/input/coatlitemedium-384-exp1/{F}.pth"
    st = torch.load(path, map_location='cpu')
    
    model = Model4(num_classes=10, seg_classes=3, arch='medium', mask_head=False)
    model.load_state_dict(st)
    model.cuda()
    model.eval()
    
    classification_models.append(copy.deepcopy(model))
    

    
for F in [123123, 123123123]:
    path = f"/kaggle/input/rsna-abd-v2s-try5-v10-fulldata/{F}.pth"
    st = torch.load(path, map_location='cpu')
   
    model = Model3(n_classes=10)
    model.load_state_dict(st)
    model.cuda()
    model.eval()

    classification_models.append(copy.deepcopy(model))
    
#------------------------------------------------------------------------
#------------------------------------------------------------------------
#------------------------------------------------------------------------
#'''





#------------------------------------------------------------------------
#------------------------------------------------------------------------
#------------------------------------------------------------------------

for F in [7]:
    path = f"/kaggle/input/coat-lite-medium-bs2-lr9e5-seed7/0.pth"
    st = torch.load(path, map_location='cpu')
   
    model = Model5(n_classes=10, arch='medium', mask_head=False)
    model.load_state_dict(st)
    model.cuda()
    model.eval()

    classification_models.append(copy.deepcopy(model))

for F in [7777]:
    path = f"/kaggle/input/coat-lite-medium-bs2-lr10e5-seed7777/0.pth"
    st = torch.load(path, map_location='cpu')
   
    model = Model5(n_classes=10, arch='medium', mask_head=False)
    model.load_state_dict(st)
    model.cuda()
    model.eval()

    classification_models.append(copy.deepcopy(model))

for F in [7777777]:
    path = f"/kaggle/input/coat-lite-medium-bs2-lr11e5-seed7777777/0.pth"
    st = torch.load(path, map_location='cpu')
   
    model = Model5(n_classes=10, arch='medium', mask_head=False)
    model.load_state_dict(st)
    model.cuda()
    model.eval()

    classification_models.append(copy.deepcopy(model))
    
#for F in [42]:
#    path = f"/kaggle/input/coat-lite-medium-bs2-lr12e5-seed42/0.pth"
#    st = torch.load(path, map_location='cpu')
   
#    model = Model5(n_classes=10, arch='medium', mask_head=False)
#    model.load_state_dict(st)
#    model.cuda()
#    model.eval()

#    classification_models.append(copy.deepcopy(model))
    
#------------------------------------------------------------------------

for F in [0]:
    path = f"/kaggle/input/coat-lite-medium-bs2-lr125e6/{F}.pth"
    st = torch.load(path, map_location='cpu')
    
    model = Model5(num_classes=10, arch='medium', mask_head=False)
    model.load_state_dict(st)
    model.cuda()
    model.eval()
    
    classification_models.append(copy.deepcopy(model))

    
for F in [3]:
    path = f"/kaggle/input/coat-lite-medium-unet-bs1-lr10e5-seed7777/{F}.pth"
    st = torch.load(path, map_location='cpu')
    
    model = Model7(num_classes=4, arch='medium', mask_head=False)
    model.load_state_dict(st)
    model.cuda()
    model.eval()
    
    classification_models.append(copy.deepcopy(model))
    
    
for F in [69, 17716124]:
    path = f"/kaggle/input/rsna-abd-models-2/try11_cls_tf_efficientnetv2_s_in21ft1k_v1_fulldata/{F}.pth"
    st = torch.load(path, map_location='cpu')
   
    model = Model3(n_classes=10)
    model.load_state_dict(st)
    model.cuda()
    model.eval()

    classification_models.append(copy.deepcopy(model))
    
#------------------------------------------------------------------------
#------------------------------------------------------------------------
#------------------------------------------------------------------------
#'''



print(len(classification_models))



extra_models = []

for F in [0,1,2, 3]:
    path = f"/kaggle/input/coatsmall384extravast4funet/{F}.pth"
    st = torch.load(path, map_location='cpu')
    
    model = Model4(num_classes=2, seg_classes=4, arch='small', mask_head=False)
    model.load_state_dict(st)
    model.cuda()
    model.eval()
    
    extra_models.append(copy.deepcopy(model))
    
    #break
for F in [2024]:
    path = f"/kaggle/input/fullextracoatsmall384/3_best.pth"
    st = torch.load(path, map_location='cpu')
    
    model = Model4(num_classes=2, seg_classes=4, arch='small', mask_head=False)
    model.load_state_dict(st)
    model.cuda()
    model.eval()
    
    extra_models.append(copy.deepcopy(model))
    
# for F in [2717]:
#     path = f"/kaggle/input/fullextracoatsmall384/2_best.pth"
#     st = torch.load(path, map_location='cpu')
    
#     model = Model4(num_classes=2, seg_classes=4, arch='small', mask_head=False)
#     model.load_state_dict(st)
#     model.cuda()
#     model.eval()
    
#     extra_models.append(copy.deepcopy(model))  
    
    
print(len(extra_models))




#now this is a list only for name
extra_models_theo_newseg = []

for F in [0,1,2,3]:
    path = f"/kaggle/input/rsna-abd-try11-v8-extrav/{F}_best.pth"
    st = torch.load(path, map_location='cpu')
    
    model = Model6(n_classes=2)
    model.load_state_dict(st)
    model.cuda()
    model.eval()
    
    extra_models_theo_newseg.append(copy.deepcopy(model))
    
    #break

for F in [0, 2]:
    path = f"/kaggle/input/coatmedium384extravast/{F}_best.pth"
    st = torch.load(path, map_location='cpu')
    
    model = Model5(n_classes=2)
    model.head = nn.Sequential(
            #nn.Linear(lstm_embed, lstm_embed//2),
            #nn.BatchNorm1d(lstm_embed//2),
            nn.Dropout(0.1),
            #nn.LeakyReLU(0.1),
            nn.Linear(512*2, 2),
        )
    model.load_state_dict(st)
    model.cuda()
    model.eval()
    
    extra_models_theo_newseg.append(copy.deepcopy(model))
    
#     extra_models_theo_newseg.append(copy.deepcopy(model))
    
'''
for F in range(4):
    path = f"/kaggle/input/rsna-abd-coat-small-extrav/{F}_best.pth"
    st = torch.load(path, map_location='cpu')
    
    model = Model6(n_classes=2)
    model.load_state_dict(st)
    model.cuda()
    model.eval()
    
    extra_models_theo_newseg.append(copy.deepcopy(model))
    
    #break
'''

print('theo extra:, ', len(extra_models_theo_newseg))

import torch.nn.functional as F


PATIENT_TO_PREDICTION = {}
PATIENT_TO_PREDICTION2 = {}

#if local_rank==0:
#    test_patients = [58359,]
#else:
#    test_patients = [31966]

PATIENT_TO_PREDICTION = {}
PATIENT_TO_PREDICTION2 = {}
PATIENT_TO_PREDICTION_THEO = {}
PATIENT_TO_PREDICTION4 = {}


if local_rank==0:
    test_patients = test_patients[:len(test_patients)//2]
    #test_patients = test_patients[:5]
else:
    test_patients = test_patients[len(test_patients)//2:]
    #test_patients = test_patients[:10]


for patient in tqdm(test_patients):
#for patient in tqdm(test_patients[1+local_rank:]):
    try:
    #if 1:

        studies = os.listdir(f'{IMAGE_FOLDER}/{patient}')

        final_outputs = []
        final_outputs2 = []
        final_outputs3 = []
        final_outputs4 = []
        for study in studies:

            files = glob_sorted(f"{IMAGE_FOLDER}/{patient}/{study}/*")

            #volume = load_volume(files)
            #volume_theo, files_theo = load_volume_theo(files)
            
            volume, file_to_volume_theo, files_theo = load_volume(files)
            
            file_to_volume = {file: vol for file, vol in zip(files, volume)}

            volumes = process_volume(volume)
            volumes_seg = predict_segmentation(volumes, segmentation_models)
            volume_seg = np.concatenate(volumes_seg.transpose(0, 2, 1, 3, 4))[:len(volume)]

            msk = volume_seg.max(0).max(0)
            ys, xs = np.where(msk)
            #y1, y2, x1, x2 = np.min(ys) / 128, np.max(ys) / 128, np.min(xs) / 128, np.max(xs) / 128
            y1, y2, x1, x2 = np.min(ys) / 128, np.max(ys) / 128, np.min(xs) / 128, np.max(xs) / 128
    
            files = pd.DataFrame({"file": files})
            files_theo = pd.DataFrame({"file": files_theo})

            files_volumes = get_volume_data(files, step=96, stride=2, stride_cutoff=400)
            files_volumes_theo = get_volume_data(files_theo, step=96, stride=2, stride_cutoff=400)

            first = True
            
            import gc
            del volumes, volumes_seg, volume_seg, volume
            gc.collect()

            for file_volume, file_volume_theo in zip(files_volumes, files_volumes_theo):
                #volume = load_volume(file_volume.file)
                volume = np.stack([file_to_volume[file] for file in file_volume.file])
                volume_theo = np.stack([file_to_volume_theo[file] for file in file_volume_theo.file])

                if first:
                    h, w = volume.shape[1:]
                    y1, y2, x1, x2 = int(y1*h), int(y2*h), int(x1*w), int(x2*w)
                volume2 = volume
                volume_theo_uncrop = volume_theo
                volume = volume[:, y1:y2, x1:x2]
                
                #----------------------------
                #----------------------------
                #MASSIVE CHANGE HERE, NOT CROPPPING THEO DATA BECAUSE ONLY USING IT FOR EXTRAVASATION, MAKE VOLUME_THEO_UNCROP SEPERATELY WHEN USING AT BOTH PLACES
                volume_theo = volume_theo[:, y1:y2, x1:x2]
                #----------------------------
                #----------------------------
                
                #'''
                vols = []
                NC = 3
                for i in range(len(volume)//NC):
                    vols.append(volume[i*NC:(i+1)*NC])
                vol = np.stack(vols, 0).transpose(0, 2, 3, 1)
                
                volume_ = []
                for image in vol:

                    #np.random.seed(CFG.literal_step)
                    #random.seed(CFG.literal_step)
                    
                    image = image.astype(np.float32) / 255

                    transformed = test_augs2(image=image)

                    image = transformed['image']

                    volume_.append(image)

                volume = torch.stack(volume_).float()
                volume = volume.cuda()
                #'''
                
                #### UNCROPPED #####
                vols = []
                NC = 3
                for i in range(len(volume2)//NC):
                    vols.append(volume2[i*NC:(i+1)*NC])
                vol = np.stack(vols, 0).transpose(0, 2, 3, 1)

                volume_ = []
                for image in vol:

                    #np.random.seed(CFG.literal_step)
                    #random.seed(CFG.literal_step)
                    
                    image = image.astype(np.float32) / 255

                    transformed = test_augs(image=image)

                    image = transformed['image']

                    volume_.append(image)

                volume2 = torch.stack(volume_).float()
                volume2 = volume2.cuda()
                
                
                outputs = []
                outputs2 = []
                outputs3 = []
                outputs4 = []
                
                with torch.no_grad():
                    with torch.cuda.amp.autocast(enabled=True):

                        for model in classification_models[:7]:
                            outs = model(volume.unsqueeze(0))
                            outs = outs.float().sigmoid()
                            outputs.append(outs)

                        for model in extra_models:
                            outs = model(volume2.unsqueeze(0))[:, :, [1, 0]]
                            outs = outs.float().sigmoid()
                            outputs2.append(outs)
                        
#                         for model in extra_models_theo_newseg[4:]:
#                             outs = model(volume2.unsqueeze(0))[:, :, [1, 0]]
#                             outs = outs.float().sigmoid()
#                             outputs2.append(outs)
                
                
                import gc
                #del volume, volume2, volume_, vol, vols
                del volume2, volume_, vol, vols
                gc.collect()
                
                torch.cuda.empty_cache()
                
                #### THEO DATA #####
                vols = []
                NC = 3
                for i in range(len(volume_theo)//NC):
                    vols.append(volume_theo[i*NC:(i+1)*NC])
                vol = np.stack(vols, 0).transpose(0, 2, 3, 1)
                
                volume_ = []
                for image in vol:
                    #np.random.seed(CFG.literal_step)
                    #random.seed(CFG.literal_step)
                    
                    image = image.astype(np.float32) / 255

                    transformed = test_augs(image=image)
                    image = transformed['image']

                    volume_.append(image)

                volume_theo = torch.stack(volume_).float()
                volume_theo = volume_theo.cuda()
                
                with torch.no_grad():
                    with torch.cuda.amp.autocast(enabled=True):
                        for model in classification_models[7:]:
                            outs = model(volume_theo.unsqueeze(0))
                            outs = outs.float().sigmoid()
                            outputs3.append(outs)
                
                import gc
                #del volume, volume2, volume_, vol, vols
                del volume_, vol, vols
                gc.collect()
                
                torch.cuda.empty_cache()
                
                
                #### THEO UNCROP DATA #####
                vols = []
                NC = 3
                for i in range(len(volume_theo_uncrop)//NC):
                    vols.append(volume_theo_uncrop[i*NC:(i+1)*NC])
                vol = np.stack(vols, 0).transpose(0, 2, 3, 1)
                
                volume_ = []
                for image in vol:
                    #np.random.seed(CFG.literal_step)
                    #random.seed(CFG.literal_step)
                    
                    image = image.astype(np.float32) / 255

                    transformed = test_augs(image=image)
                    image = transformed['image']

                    volume_.append(image)

                volume_theo = torch.stack(volume_).float()
                volume_theo = volume_theo.cuda()
                
                with torch.no_grad():
                    with torch.cuda.amp.autocast(enabled=True):
                        
                        for model in extra_models_theo_newseg[:4]:
                            outs = model(volume_theo.unsqueeze(0))
                            outs = outs.float().sigmoid()
                            outputs4.append(outs)
                            
                        for model in extra_models_theo_newseg[4:]:
                            outs = model(volume_theo.unsqueeze(0))[:,:,[1,0]]
                            outs = outs.float().sigmoid()
                            outputs4.append(outs)
                
                
                outputs = torch.stack(outputs)[:, 0].mean(0)
                outputs2 = torch.stack(outputs2)[:, 0].mean(0)
                outputs3 = torch.stack(outputs3)[:, 0].mean(0)
                outputs4 = torch.stack(outputs4)[:, 0].mean(0)
                
                #outputs = (outputs * 0.5) + (outputs3 * 0.5)
                #outputs = outputs3
                
                final_outputs.append(outputs.detach().cpu().numpy())
                final_outputs2.append(outputs2.detach().cpu().numpy())
                final_outputs3.append(outputs3.detach().cpu().numpy())
                final_outputs4.append(outputs4.detach().cpu().numpy())

                first = False
                
                torch.cuda.empty_cache()

#               break
    except:
#     else:
        final_outputs = last_final_outputs.copy()
        final_outputs2 = last_final_outputs2.copy()
        final_outputs3 = last_final_outputs3.copy()
        final_outputs4 = last_final_outputs4.copy()
    
    last_final_outputs = final_outputs.copy()
    last_final_outputs2 = final_outputs2.copy()
    last_final_outputs3 = final_outputs3.copy()
    last_final_outputs4 = final_outputs4.copy()

    final_outputs = np.concatenate(final_outputs)
    final_outputs2 = np.concatenate(final_outputs2)
    final_outputs3 = np.concatenate(final_outputs3)
    final_outputs4 = np.concatenate(final_outputs4)

    final_predictions = final_outputs.max(0)
    final_predictions2 = final_outputs2.max(0)
    final_predictions3 = final_outputs3.max(0)
    final_predictions4 = final_outputs4.max(0)

    PATIENT_TO_PREDICTION[patient] = final_predictions
    PATIENT_TO_PREDICTION2[patient] = final_predictions2
    PATIENT_TO_PREDICTION_THEO[patient] = final_predictions3
    PATIENT_TO_PREDICTION4[patient] = final_predictions4

    if DEBUG:
        break
        
np.save(f'PATIENT_TO_PREDICTION_rank{local_rank}.npy', PATIENT_TO_PREDICTION)
np.save(f'PATIENT_TO_PREDICTION2_rank{local_rank}.npy', PATIENT_TO_PREDICTION2)
np.save(f'PATIENT_TO_PREDICTION_THEO_rank{local_rank}.npy', PATIENT_TO_PREDICTION_THEO)
np.save(f'PATIENT_TO_PREDICTION4_rank{local_rank}.npy', PATIENT_TO_PREDICTION4)

sys.exit()

Writing ddp.py


In [3]:
!CUDA_VISIBLE_DEVICES=0,1, python -m torch.distributed.launch --nproc_per_node=2 ddp.py

/opt/conda/lib/python3.10/site-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/kaggle/working/./pretrainedm

In [4]:
# 9 coat med + 8 v2s + 4 coat small + 3 v2s + 1 coat med 

In [5]:
import numpy as np

PATIENT_TO_PREDICTION_THEO = np.load('/kaggle/working/PATIENT_TO_PREDICTION_THEO_rank0.npy', allow_pickle=True).item()
PATIENT_TO_PREDICTION_THEO_rank1 = np.load('/kaggle/working/PATIENT_TO_PREDICTION_THEO_rank1.npy', allow_pickle=True).item()
PATIENT_TO_PREDICTION_THEO.update(PATIENT_TO_PREDICTION_THEO_rank1)

PATIENT_TO_PREDICTION = np.load('/kaggle/working/PATIENT_TO_PREDICTION_rank0.npy', allow_pickle=True).item()
PATIENT_TO_PREDICTION_rank1 = np.load('/kaggle/working/PATIENT_TO_PREDICTION_rank1.npy', allow_pickle=True).item()
PATIENT_TO_PREDICTION.update(PATIENT_TO_PREDICTION_rank1)

PATIENT_TO_PREDICTION2 = np.load('/kaggle/working/PATIENT_TO_PREDICTION2_rank0.npy', allow_pickle=True).item()
PATIENT_TO_PREDICTION2_rank1 = np.load('/kaggle/working/PATIENT_TO_PREDICTION2_rank1.npy', allow_pickle=True).item()
PATIENT_TO_PREDICTION2.update(PATIENT_TO_PREDICTION2_rank1)

PATIENT_TO_PREDICTION4 = np.load('/kaggle/working/PATIENT_TO_PREDICTION4_rank0.npy', allow_pickle=True).item()
PATIENT_TO_PREDICTION4_rank1 = np.load('/kaggle/working/PATIENT_TO_PREDICTION4_rank1.npy', allow_pickle=True).item()
PATIENT_TO_PREDICTION4.update(PATIENT_TO_PREDICTION4_rank1)

In [6]:
FINAL_SUB = {'patient_id': [], 'bowel_healthy': [], 'bowel_injury': [], 
             'extravasation_healthy': [], 'extravasation_injury': [], 
             'kidney_healthy': [], 'kidney_low': [], 'kidney_high': [],
             'liver_healthy': [], 'liver_low': [], 'liver_high': [],
             'spleen_healthy': [], 'spleen_low': [], 'spleen_high': [],}

for patient in PATIENT_TO_PREDICTION:
    prediction = PATIENT_TO_PREDICTION[patient].copy()
    prediction2 = PATIENT_TO_PREDICTION2[patient].copy()
    prediction3 = PATIENT_TO_PREDICTION_THEO[patient].copy()
    prediction4 = PATIENT_TO_PREDICTION4[patient].copy()
    
    #ensemble organ model
    #prediction = (prediction + prediction3) / 2
    prediction = (prediction * 0.5) + (prediction3 * 0.5)
    
    #bowel avg from extrav-bowel
    prediction[9] = (prediction[9] * 0.7) + (prediction4[1] * 0.15) + (prediction2[1]*0.15)
    
    #ensemble extrav
    prediction2[0] = (prediction2[0] + prediction4[0]) / 2
#     prediction2[0] = prediction4[0]
    
    
    #prediction = prediction3
    
    FINAL_SUB['patient_id'].append(patient)

    #FINAL_SUB['bowel_healthy'].append(0.9796631712742294)
    #FINAL_SUB['bowel_injury'].append(0.040673657451541154)

    #FINAL_SUB['bowel_healthy'].append(1 - prediction[9])
    #FINAL_SUB['bowel_injury'].append(prediction[9] * 2)

    #FINAL_SUB['bowel_injury'] = prediction[9]*0.9 + 0.020336828725770577*0.1
    #FINAL_SUB['bowel_healthy'] = (1 - prediction[9])*0.9 + 0.9796631712742294*0.1
    
#     bowel_w = 2
#     extrav_w = 6
#     low_w = 2
#     high_w = 4
    
    bowel_w = 1.75
    extrav_w = 8
    low_w = 1.75
    high_w = 3.5
    
    FINAL_SUB['bowel_healthy'].append(1 - prediction[9])
    FINAL_SUB['bowel_injury'].append(prediction[9] * bowel_w)
    #FINAL_SUB['bowel_injury'].append(prediction[9])
    
#     FINAL_SUB['extravasation_healthy'].append(0.936447410231967*0.5 + (1-prediction2[0])*0.5)
#     FINAL_SUB['extravasation_injury'].append( (0.3813155386081983*0.5 + prediction2[0]*extrav_w) + prediction[9]) #the trick is extrav is better with extrav+bowel
    
    FINAL_SUB['extravasation_healthy'].append(1 - prediction2[0])
    FINAL_SUB['extravasation_injury'].append(0.06355258976803305 + (prediction2[0] * extrav_w))
    
#     FINAL_SUB['extravasation_healthy'].append(0.936447410231967*0.5 + (1-prediction2[0])*0.5)
#     FINAL_SUB['extravasation_injury'].append( (0.3813155386081983*0.5 + prediction2[0]*extrav_w)) #the trick is extrav is better with extrav+bowel
    
    
    FINAL_SUB['liver_healthy'].append(1 - prediction[0])
    FINAL_SUB['liver_low'].append(prediction[3]*low_w)
    FINAL_SUB['liver_high'].append(prediction[4]*high_w)

    #valid_study_level_pred['kidney_healthy'] = valid_study_level['kidney_healthy'].mean()
    #valid_study_level_pred['kidney_low'] = valid_study_level['kidney_low'].mean()
    #valid_study_level_pred['kidney_high'] = valid_study_level['kidney_high'].mean()

    FINAL_SUB['spleen_healthy'].append(1 - prediction[1])
    FINAL_SUB['spleen_low'].append(prediction[5]*low_w)
    FINAL_SUB['spleen_high'].append(prediction[6]*high_w)

    FINAL_SUB['kidney_healthy'].append(1 - prediction[2])
    FINAL_SUB['kidney_low'].append((prediction[7])*low_w)
    FINAL_SUB['kidney_high'].append(prediction[8]*high_w)

    #break

In [7]:
import pandas as pd
submission = pd.DataFrame(FINAL_SUB)
submission

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high
0,26501,0.997718,0.003993,0.998705,0.073914,0.993237,0.013022,0.002546,0.983368,0.027066,0.005253,0.992961,0.016052,0.001973
1,10051,0.994993,0.008763,0.992387,0.124455,0.989373,0.015757,0.006582,0.981912,0.029391,0.004002,0.379380,0.946607,0.330375


In [8]:
!rm -r /kaggle/working/*

In [9]:
submission.to_csv('./submission.csv', index=False)